In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import math

In [3]:
interpreter = tf.lite.Interpreter(model_path=r"C:\Users\tejat\Desktop\FitnessApp\3.tflite")
interpreter.allocate_tensors()

c:\Users\tejat\anaconda3\envs\gymbro\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [4]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [5]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [7]:
#frame_counter = 0
#interval = 5  
mini=float("inf")
maxi=float("-inf")
cap = cv2.VideoCapture(r"C:\Users\tejat\Desktop\FitnessApp\squats.mp4")

if not cap.isOpened():
    print("Error: Could not video.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Couldn't read frame.")
        break
    
    # Convert frame to RGB (for TensorFlow)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)

    # Setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Make predictions
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())  # Ensure correct format
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    keypoints=keypoints_with_scores[0,0,:,:]
    left_hip=keypoints[11]
    left_knee=keypoints[13]
    left_ankel=keypoints[15]
    right_hip=keypoints[12]
    right_knee=keypoints[14]
    right_ankel=keypoints[16]
    left_shoulder=keypoints[5]
    right_shoulder=keypoints[6]
    '''print("Left Shoulder:", left_shoulder)
    print("Left Hip:", left_hip)
    print("Left Knee:", left_knee)'''
    #print(left_hip)
    #print(keypoints_with_scores)
    # In your loop:
    cv2.circle(frame, (int(left_hip[1] * frame.shape[1]), int(left_hip[0] * frame.shape[0])), 8, (0, 0, 255), -1)  # Hip (RED)
    cv2.circle(frame, (int(left_knee[1] * frame.shape[1]), int(left_knee[0] * frame.shape[0])), 8, (255, 0, 0), -1)  # Knee (BLUE)
    cv2.circle(frame, (int(left_shoulder[1] * frame.shape[1]), int(left_shoulder[0] * frame.shape[0])), 8, (0, 255, 0), -1)  # Shoulder (GREEN)

# Print coordinates and confidence
    print(f"Shoulder: (X={left_shoulder[1]:.2f}, Y={left_shoulder[0]:.2f}, Conf={left_shoulder[2]:.2f})")
    print(f"Hip:      (X={left_hip[1]:.2f}, Y={left_hip[0]:.2f}, Conf={left_hip[2]:.2f})")
    print(f"Knee:     (X={left_knee[1]:.2f}, Y={left_knee[0]:.2f}, Conf={left_knee[2]:.2f})")
    if (left_shoulder[2] > 0.7) and (left_hip[2] > 0.7) and (left_knee[2] > 0.4):
        a1=findangle(left_shoulder,left_hip,left_knee)
    #print(a1)
        mini=min(mini,a1)
        maxi=max(maxi,a1)
    print(f"Shoulder: {left_shoulder}, Hip: {left_hip}, Knee: {left_knee}, Angle: {a1}")

    # Rendering (Assuming functions exist)
    draw_connections(frame, keypoints, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)

    # Display the frame
    cv2.imshow('MoveNet Lightning', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Adjust wait time
        break
print(maxi)
print(mini)

cap.release()
cv2.destroyAllWindows()

Error: Could not video.
-inf
inf


In [ ]:
def findangle(p1, p2, p3):
    # Convert [Y,X] to [X,Y] and scale to frame size
    p1 = np.array([p1[1], p1[0]])  # Shoulder
    p2 = np.array([p2[1], p2[0]])  # Hip
    p3 = np.array([p3[1], p3[0]])  # Knee
    
    # Vectors from hip (p2) to shoulder (p1) and knee (p3)
    v1 = p1 - p2
    v2 = p3 - p2
    
    # Dot product and magnitudes
    dot_product = np.dot(v1, v2)
    mag_v1 = np.linalg.norm(v1)
    mag_v2 = np.linalg.norm(v2)
    
    # Avoid division by zero
    if mag_v1 * mag_v2 < 1e-6:
        return 0
    
    # Clip to handle floating-point errors
    cos_theta = np.clip(dot_product / (mag_v1 * mag_v2), -1.0, 1.0)
    return np.degrees(np.arccos(cos_theta))

: 